In [1]:
import numpy as np
import pandas as pd
import pickle
import random
import json
from collections import Counter
from tqdm import tqdm

#### In this notebook, we identify model majority labels over 5 runs, for 5-class and 3-class setting.

##### Load our dataset

In [6]:
df_gold = pd.read_csv(r"../data/qualtrics_survey/results/df_gold_all.csv")

In [7]:
classes_5 = ['Unrelated', 'Consistent', 'Indirect inconsistency', "Factual inconsistency", 'Surface contradiction']                             
classes_3 = ['Unrelated', 'Consistent', 'Inconsistent']
classes_5_to_3 = {'Unrelated': 'Unrelated', 'Consistent': 'Consistent', 'Indirect inconsistency': 'Inconsistent', "Factual inconsistency": 'Inconsistent', 'Surface contradiction': 'Inconsistent'}

### Identify majority (3 classes)

In [ ]:
df_gold['LLaMA 8B majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['LLaMA 8B 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['LLaMA 8B majority 3 class'][i] = majority
    
    

In [ ]:
df_gold['LLaMA 70B majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['LLaMA 70B 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['LLaMA 70B majority 3 class'][i] = majority
    
    

In [ ]:
df_gold['ChatGPT-3.5 majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['ChatGPT-3.5 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-3.5 majority 3 class'][i] = majority

In [ ]:
df_gold['ChatGPT-4 majority 3 class'] = 0
for i in range(len(df_gold)):
    list_answers = eval(df_gold['ChatGPT-4 5 runs'][i])
    list_answers = [classes_5_to_3[el] for el in list_answers]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-4 majority 3 class'][i] = majority


### Identify majority (5 classes)

In [ ]:
df_gold['LLaMA 8B majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['LLaMA 8B 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['LLaMA 8B majority'][i] = majority
    
    

In [ ]:
df_gold['LLaMA 70B majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['LLaMA 70B 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['LLaMA 70B majority'][i] = majority
    
    

In [ ]:
df_gold['ChatGPT-3.5 majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['ChatGPT-3.5 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-3.5 majority'][i] = majority


In [ ]:
df_gold['ChatGPT-4 majority'] = 0
for i in range(len(df_gold)):
    list_answers = df_gold['ChatGPT-4 5 runs'][i]
    counter = Counter(list_answers)
    max_count = max(counter.values())
    top_labels = [label for label, count in counter.items() if count == max_count]
    majority = random.choice(top_labels)
    df_gold['ChatGPT-4 majority'][i] = majority

In [ ]:
df_gold.to_csv(r"../data/qualtrics_survey/results/df_gold_all.csv", index='False')